In [2]:
import pandas as pd
from preprocess import preprocess_tweet
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from string import punctuation, digits

from sklearn.feature_extraction.text import (
    CountVectorizer, TfidfVectorizer)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import (
    SelectKBest, VarianceThreshold, f_classif)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
from sklearn.manifold import TSNE
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D,concatenate,Flatten,\
Dense,Dropout,LSTM,Masking,Bidirectional,Dropout,GRU,SimpleRNN,TimeDistributed, BatchNormalization, Activation, MaxPooling1D, GlobalMaxPooling1D, Conv1D
from keras.models import Sequential,Model

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

from string import punctuation
from os import listdir
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.core import Reshape, Flatten
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers import Embedding

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras import regularizers


from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# Any results you write to the current directory are saved as output.
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.layers.convolutional import Conv1D
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models import Word2Vec

from gensim.models.keyedvectors import KeyedVectors


## Load data

In [44]:
n_data = []
p_data = []

# read positive and negative tweet data
# use own path here
with open('/Users/louancillon/Documents/ETHZ/M2/CIL/ETH_CIL22/twitter-datasets/train_neg_full.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    n_data = lines
print('read negative tweet lines ', len(n_data))

with open('/Users/louancillon/Documents/ETHZ/M2/CIL/ETH_CIL22/twitter-datasets/train_pos_full.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    p_data = lines
print('read positive tweet lines ', len(p_data))

# fill all tweet data and target labels
tweet_data = p_data + n_data
tweet_labels = [1.0 for _ in p_data] + [0.0 for _ in n_data]
print(len(tweet_data))
print(len(tweet_labels))


read negative tweet lines  1250000
read positive tweet lines  1250000
2500000
2500000


In [45]:
df = pd.DataFrame({'review':tweet_data, 'sentiment':tweet_labels})

In [46]:
# apply the above function to df['text']
#prep_tweet = df['review'].map(lambda x: preprocess_tweet(x, False))
original = df['review'].map(lambda x: x.split())

In [47]:
#df.insert(1, 'preprocessed',prep_tweet)
df.insert(1, 'original',original)

In [48]:
df.head()

,review,original,sentiment
0,<user> i dunno justin read my mention or not ....,"[<user>, i, dunno, justin, read, my, mention, ...",1.0
1,"because your logic is so dumb , i won't even c...","[because, your, logic, is, so, dumb, ,, i, won...",1.0
2,""" <user> just put casper in a box ! "" looved t...","["", <user>, just, put, casper, in, a, box, !, ...",1.0
3,<user> <user> thanks sir > > don't trip lil ma...,"[<user>, <user>, thanks, sir, >, >, don't, tri...",1.0
4,visiting my brother tmr is the bestest birthda...,"[visiting, my, brother, tmr, is, the, bestest,...",1.0


In [49]:
#X_train, X_test,Y_train, Y_test = train_test_split(df['preprocessed'], df['sentiment'], test_size=0.2, random_state = 45)
X_train, X_test,Y_train, Y_test = train_test_split(df['original'], df['sentiment'], test_size=0.1, random_state = 45)


In [50]:
model = Word2Vec(X_train, size=150, window=10, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

# save model in ASCII (word2vec) format
model.save("W2V_150_10")

Vocabulary size: 554764


In [51]:
# Set Maximum number of words to be embedded
NUM_WORDS = 25000

# Define/Load Tokenize text function
#tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
tokenizer = Tokenizer(num_words=NUM_WORDS,lower=True)

# Fit the function on the text
tokenizer.fit_on_texts(X_train)

# Count number of unique tokens
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 554764 unique tokens.


In [52]:
# Convert train and val to sequence
sequences_train = tokenizer.texts_to_sequences(X_train)

In [53]:
# Limit size of train/val to 50 and pad the sequence
X_train = pad_sequences(sequences_train,maxlen=50)

# Convert target to array
Y_train = np.asarray(Y_train)

# Printing shape
print('Shape of X train and X validation tensor:', X_train.shape)
print('Shape of label train and validation tensor:', Y_train.shape)

Shape of X train and X validation tensor: (2250000, 50)
Shape of label train and validation tensor: (2250000,)


## Embedding  

In [54]:
#model = Word2Vec.load("W2V_150_10")
word_vectors = model.wv


In [55]:
EMBEDDING_DIM=150
vocabulary_size=min(len(word_index)+1,(NUM_WORDS))

embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))


for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        vec = np.zeros(EMBEDDING_DIM)
        if word in bad_words:
            vec = word_vectors['fuck']
        embedding_matrix[i]=vec

In [56]:
# Dfine Embedding function using the embedding_matrix
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [57]:
sequence_length = X_train.shape[1]
filter_sizes = [3,4]
num_filters = 512
drop = 0.2

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)

merged_tensor = concatenate([maxpool_0, maxpool_1], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((2*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
conc = Dense(1)(dropout)
output = Dense(units=1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.001))(conc)

# this creates a model that includes
model = Model(inputs, output)

In [58]:
file_path =  "modelstack1024.h5"
checkpoint = ModelCheckpoint(file_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="acc", mode="max", patience=5, verbose=1)
red = ReduceLROnPlateau(monitor="acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint,early, red]

model.compile(optimizer=optimizers.Adam(0.001), loss=losses.binary_crossentropy, metrics=['acc'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 150)      3750000     input_6[0][0]                    
__________________________________________________________________________________________________
reshape_10 (Reshape)            (None, 50, 150, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 48, 1, 512)   230912      reshape_10[0][0]                 
____________________________________________________________________________________________

In [ ]:
# Compiling Model using optimizer
#opt = optimizers.Adam(0.001)
#model.compile(loss='binary_crossentropy',optimizer=opt, metrics = ['accuracy'])

# Fitting Model to the data
#callbacks = [EarlyStopping(monitor='accuracy')]
model.fit(X_train, Y_train, batch_size=1024, epochs=100, verbose=1)  # starts training

Epoch 1/100
2198/2198 [==============================] - 2248s 1s/step - loss: 0.4167 - acc: 0.8313
Epoch 2/100
2198/2198 [==============================] - 2211s 1s/step - loss: 0.3831 - acc: 0.8444
Epoch 3/100
2198/2198 [==============================] - 2208s 1s/step - loss: 0.3713 - acc: 0.8484
Epoch 4/100
2198/2198 [==============================] - 2207s 1s/step - loss: 0.3641 - acc: 0.8512
Epoch 5/100
2198/2198 [==============================] - 2203s 1s/step - loss: 0.3596 - acc: 0.8527
Epoch 6/100
2198/2198 [==============================] - 2193s 997ms/step - loss: 0.3563 - acc: 0.8541
Epoch 7/100
2198/2198 [==============================] - 2184s 994ms/step - loss: 0.3542 - acc: 0.8550
Epoch 8/100
2198/2198 [==============================] - 2175s 990ms/step - loss: 0.3516 - acc: 0.8561
Epoch 9/100
2198/2198 [==============================] - 2173s 989ms/step - loss: 0.3496 - acc: 0.8571
Epoch 10/100
2198/2198 [==============================] - 2168s 986ms/step - loss: 0.348

In [ ]:
# Convert train and val to sequence
sequences_test = tokenizer.texts_to_sequences(X_test)
# Limit size of train/val to 50 and pad the sequence
X_test = pad_sequences(sequences_test,maxlen=50)
# Convert target to array
Y_test = np.asarray(Y_test)

In [ ]:
# evaluate
loss, acc = model.evaluate(X_test, Y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [73]:
# Convert train and val to sequence
sequences_test = tokenizer.texts_to_sequences(test['test'])
# Limit size of train/val to 50 and pad the sequence
X_test = pad_sequences(sequences_test,maxlen=50)


In [74]:
test_pred = model.predict(X_test)

In [106]:
test_preds = [1 if test_pred[i]>0.5 else 0 for i in range(len(test_pred))]

In [28]:
model.save("modelstackembfull0.008.h5")

In [21]:
model = models.load_model("modelstackembfull0.008.h5")

In [30]:
pred = model.predict(X_test)

### Embedding 

In [8]:
filename = "embedding_word2vec_full0.1.txt"

In [39]:
model = Word2Vec(X_train, size=100, window=5, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

# save model in ASCII (word2vec) format
model.save(filename)

Vocabulary size: 107402


In [9]:
# load embedding from file
model = Word2Vec.load(filename)

In [10]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(model):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(model.wv.vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, 42, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    i=0
    for word in model.wv.vocab.keys():
        #weight_matrix[i] = model[word].wv.__getitem__()
        weight_matrix[i] = model.wv.__getitem__(word)
        i+=1
    return weight_matrix

In [11]:
def sentence_embedd_concatenate(model,data, original):
    x = []
    maxwords = max([len(original[i]) for i in range(len(original))])
    for index in range(len(data)):
        # remove out-of-vocabulary words
        doc = [word for word in data.iloc[index] if word in model.wv.vocab.keys()]
        
        if not doc:
            # append zero vector
            x.append(np.zeros((maxwords,100)))

        else:
            # append the average vector for each sentence
            sentence = np.array(model.wv[doc])
            #x.append(np.array(model[doc], dtype=object))
            #print(len(model[doc]))
            if len(model.wv.__getitem__(doc)) < maxwords:
                sentence = np.append(sentence, np.zeros(((maxwords-len(model.wv.__getitem__(doc))),100)), axis=0)
                #print(len(sentence))
            x.append(np.array(sentence))
            
        
    X = np.array(x)
    return X

In [ ]:
x_train = sentence_embedd_concatenate(model, X_train, original)
x_test = sentence_embedd_concatenate(model, X_test, original)

## Model 

In [ ]:
# define model
model5 = Sequential()
#model5.add(embedding_layer)
model5.add(Conv1D(filters=256, kernel_size=5, activation='relu', input_shape=(64, 100)))
model5.add(MaxPooling1D(pool_size=2))
model5.add(BatchNormalization())
model5.add(Dropout(0.3))
model5.add(Dense(64,activation='relu'))
model5.add(BatchNormalization())
model5.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
model5.add(Dropout(0.3))
model5.add(Flatten())
model5.add(Dense(1, activation='sigmoid'))

file_path =  "model_emb.h5"
checkpoint = ModelCheckpoint(file_path, monitor='acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="acc", mode="max", patience=5, verbose=1)
red = ReduceLROnPlateau(monitor="acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint,early, red]

model5.compile(optimizer=optimizers.Adam(0.001), loss=losses.binary_crossentropy, metrics=['acc'])
model5.summary()


# fit network
model5.fit(x_train, Y_train, epochs=4, verbose=2)
# evaluate
loss, acc = model5.evaluate(x_test, Y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))
model5.save(file_path)

## Load models 

In [15]:
## Same model on the original data without preprocessing 80%training : 82.5 accuracy 
## Same model on the original data without preprocessing 90%training : 83.02 accuracy 

model_stack = models.load_model("modelstack256x2.h5")
loss, acc = model_stack.evaluate(x_test, Y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 82.505000
